# Create validation dataset

This script creates a fixed-size validation dataset of discharge notes for a heart failure cohort 
extracted from MIMIC-IV. It samples patients, extracts structured note sections (Chief Complaint, 
HPI, Physical Exam), and stores the processed data in JSON for downstream manual annotation and LLM evaluation.

Workflow:
---------
1. Connects to PostgreSQL (MIMIC-IV) and selects 50 random patients with ICD-10 codes for heart failure.
2. Selects 50 discharge notes (one per admission).
3. Extracts key sections from each note using regex (Chief Complaint, HPI, Physical Exam).
4. Prepares a structured JSON with placeholders for model outputs.
5. Later updates the file to ensure full note text is included (for CC extraction).

Output:
-------
- 'groud_truth.json' – initial structured note fragments + metadata.
- 'groud_truth_and_full_notes.json' – updated with full note text and re-parsed CC for completeness.

PD: The plan was to make this reproductible but the notes sent by postgres are different each time  

In [ ]:
# === Install dependecies and conect to PostgreSQL database ===

from sqlalchemy import create_engine
from sqlalchemy import text
from collections import OrderedDict
import pandas as pd
import regex as re
import json
DB_NAME = "mimic"
DB_USER = "postgres"
DB_PASSWORD = "sd98hS&GD3F4"
DB_HOST = "localhost"
DB_PORT = "5432"

engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [ ]:
# === Get the heart failure cohort and select 50 random patients ===
def get_cohort(icd_code):
    query = text('''
    SELECT DISTINCT subject_id
    FROM mimiciv_hosp.diagnoses_icd
    WHERE icd_code_v2 LIKE :icd_code
    ORDER BY subject_id;
    ''')               
    return pd.read_sql(query, engine, params={"icd_code": icd_code})

cohort = get_cohort("I50%") 
sampled_subjects = cohort.sample(n=50, random_state = 42) # 42 = fixed seed for reproducibility
print(sampled_subjects.size)
subject_ids = sampled_subjects['subject_id'].tolist()

# === Get the admissions with notes for the sampled patients and select 50 random admissions ===
query = text('''
SELECT hadm_id
FROM mimiciv_note.discharge
WHERE subject_id = ANY(:subject_ids)
''')
admissions = pd.read_sql(query, engine, params={"subject_ids": subject_ids})
sampled_admissions = admissions.sample(n=50, random_state = 42)
print(sampled_admissions.size)
hadm_ids = sampled_admissions['hadm_id'].tolist()

# === Get the notes for the sampled admissions ===
query = text('''
SELECT ROW_NUMBER() OVER (ORDER BY hadm_id, charttime) AS note_number,
    note_id, subject_id, hadm_id, charttime, text
FROM mimiciv_note.discharge
WHERE text IS NOT NULL
  AND hadm_id = ANY(:hadm_ids)
''')
notes = pd.read_sql(query, engine, params={"hadm_ids": hadm_ids})
print(notes["text"].size)

# print(notes["text"][0])
# print(notes.head())

50
50
50


In [ ]:
# === Extract Chief Complaint, Past Medical History and Physical Exam sections ===
def extract_sections(note):
    
    # === Define the pattern to match the desired section ===  
    sections = {
        "Chief Complaint": [
            ("Chief Complaint:", "Major Surgical or Invasive Procedure:"),
            ("___ Complaint:", "Major Surgical or Invasive Procedure:")
        ],
        "History of Present Illness": [
            ("History of Present Illness:", "Past Medical History:"),
            ("___ Present Illness:", "Past Medical History:")
        ],
        "Physical Exam": [
            ("Physical Exam:", "Pertinent Results:"),
            ("Physical ___:", "Pertinent Results:"),
            ("Physical Exam:", "Brief Hospital Course:")
        ]
    }

    extracted_sections = {}
    for section_name, marker_pairs in sections.items():
        found = False
        for start_marker, end_marker in marker_pairs:
            if start_marker in note and end_marker in note:
                pattern = re.escape(start_marker) + r"\s*(.*?)\s*" + re.escape(end_marker)
                match = re.search(pattern, note, re.DOTALL)
                if match:
                    extracted_sections[section_name] = match.group(1).strip()
                    found = True
                    break
        if not found:
            extracted_sections[section_name] = "Section not found."
    return extracted_sections

# === Define the template ===
def create_note_template(row):
    return {
        "note_number": int(row["note_number"]),
        "note_id": str(row["note_id"]),
        "subject_id": int(row["subject_id"]),
        "hadm_id": int(row["hadm_id"]),
        "text": row["text"],
        "note_fragment": str(row["note_fragment"]),
        "chief_complaint": str(row["chief_complaint"]),
        "extracted_data": {
            "Emergency Department": {
                "Body Temperature": None,
                "Systolic Blood Pressure": None,
                "Diastolic Blood Pressure": None,
                "Heart Rate": None,
                "Respiratory Rate": None,
                "Oxygen Saturation": None
            },
            "Admission": {
                "Body Temperature": None,
                "Systolic Blood Pressure": None,
                "Diastolic Blood Pressure": None,
                "Heart Rate": None,
                "Respiratory Rate": None,
                "Oxygen Saturation": None
            },
            "Transfer": {
                "Body Temperature": None,
                "Systolic Blood Pressure": None,
                "Diastolic Blood Pressure": None,
                "Heart Rate": None,
                "Respiratory Rate": None,
                "Oxygen Saturation": None
            },
            "Discharge": {
                "Body Temperature": None,
                "Systolic Blood Pressure": None,
                "Diastolic Blood Pressure": None,
                "Heart Rate": None,
                "Respiratory Rate": None,
                "Oxygen Saturation": None
            },
            "Unknown": {
                "Body Temperature": None,
                "Systolic Blood Pressure": None,
                "Diastolic Blood Pressure": None,
                "Heart Rate": None,
                "Respiratory Rate": None,
                "Oxygen Saturation": None
            }
        }
    }

In [ ]:
# === Parse the notes ===
sampled_notes = notes

for index, row in notes.iterrows():

    sections = extract_sections(row["text"])
    cc = sections["Chief Complaint"]
    hpi = sections["History of Present Illness"]
    pe = sections["Physical Exam"]

    note_fragment = f"History of Present Illness:\n{hpi}\n\n---------------------------\n\nPhysical Exam:\n{pe}"
    sampled_notes.loc[index, "note_fragment"] = note_fragment
    sampled_notes.loc[index, "chief_complaint"] = cc

# print(notes.loc[0, "text"])

In [ ]:
# === Save everything to JSON file ===

json_data = [create_note_template(row) for _, row in sampled_notes.iterrows()]

with open("ground_truth.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=2)

At first I forgot to add the full notes (to extract cc), so I added them later

In [ ]:
# === Connect to the database ===
DB_NAME = "mimic"
DB_USER = "postgres"
DB_PASSWORD = "sd98hS&GD3F4"
DB_HOST = "localhost"
DB_PORT = "5432"
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# === Read the contents of the Ground Truth file ===
with open("groud_truth.json", 'r') as file:
    validation_data = json.load(file)

updated_data = []

for row in validation_data:
    
    new_record = OrderedDict()
    
    note_id = row["note_id"]

    query = text('''
    SELECT text
    FROM mimiciv_note.discharge
    WHERE note_id = :note_id
    ''')
    note = pd.read_sql(query, engine, params={"note_id": note_id})
    note  = note["text"].values[0]      

    sections = extract_sections(note)
    cc = sections["Chief Complaint"]
    hpi = sections["History of Present Illness"]
    pe = sections["Physical Exam"]
    note_fragment = f"History of Present Illness:\n{hpi}\n\n---------------------------\n\nPhysical Exam:\n{pe}"

    for key, value in row.items():
        new_record[key] = value
        
        if key == "text":
            new_record["text"] = note
            new_record["note_fragment"] = note_fragment

    new_record["chief_complaint"] = cc

    updated_data.append(new_record)

# === Save the updated list back to file ===
with open("ground_truth_and_full_notes.json", "w", encoding="utf-8") as f:
    json.dump(updated_data, f, indent=4)